In [21]:
import json
import ast
import pandas as pd
import os
import regex

### Read comments from json files

In [22]:
# Define the directory path
directory = "data/reviews"

# Initialize an empty list to store the data
data = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        file_path = os.path.join(directory, filename)
        # Read the JSON file and append its contents to the data list
        with open(file_path, "r") as file:
            file_content = file.read()
            try:
                # Try to parse the content as JSON
                json_data = json.loads(file_content)
                data.append(json_data)
            except json.JSONDecodeError:
                try:
                    # If parsing as JSON fails, try to evaluate the content as a Python dictionary
                    dict_data = ast.literal_eval(file_content)
                    # Convert the dictionary to a JSON string
                    json_data = json.dumps(dict_data)
                    data.append(json.loads(json_data))
                except (ValueError, SyntaxError) as e:
                    print(f"Error parsing JSON from file {file_path}: {e}")

# Create a dataframe from the data list
df = pd.DataFrame(data)

### Remove Unnecessary Columns

In [23]:
df.columns

Index(['id', 'title', 'content', 'status', 'thank_count', 'score', 'new_score',
       'customer_id', 'comment_count', 'rating', 'images', 'thanked',
       'created_at', 'created_by', 'suggestions', 'attributes',
       'product_attributes', 'spid', 'is_photo', 'seller', 'product_id',
       'vote_attributes', 'delivery_rating', 'timeline'],
      dtype='object')

In [24]:
kept_cols_lv1 = ['id', 'title', 'content', 'status', 'rating', 
             'suggestions', 'vote_attributes', 'delivery_rating']

df = df[kept_cols_lv1]
df = df[df.status == 'approved']
df = df[df.content != ""]

kept_cols_lv2 = ['id', 'title', 'content', 'rating']
df = df[kept_cols_lv2]

In [25]:
df

,id,title,content,rating
0,9724102,Cực kì hài lòng,Gôd,5
1,19582868,Cực kì hài lòng,Hàng tốt,5
2,17441069,Cực kì hài lòng,Bàn phím chất lượng như hình ảnh. Độ nãy tốt.\...,5
3,5373697,Rất không hài lòng,bàn phím loạn xị ngậu lên . chuột thì k sao cò...,1
4,13821612,Cực kì hài lòng,Mình thấy ảnh sản phẩm màu đen nên tưởng sẽ nh...,5
...,...,...,...,...
6510,19856146,Cực kì hài lòng,Âm thanh tốt,5
6511,15249248,Cực kì hài lòng,"Đã nhận dc đồng hồ,Tiki giao hàng và tư vấn ch...",5
6514,14210789,Cực kì hài lòng,"Chất lượng xem ra không được như mong đợi, tuy...",3
6515,19600851,Cực kì hài lòng,Âm thanh tốt. Ko bị đau tai. Chống ổn tốt.,5


#### Preprocess data

In [26]:
from utils.preprocess import (
    remove_HTML,
    convert_unicode,
    standardize_sentence_typing,
    normalize_acronyms, 
    # word_segmentation, # When use PhoBERT
    remove_unnecessary_characters,
    remove_redundant_newlines
) 

In [27]:
def text_preprocess(text):
    text = remove_HTML(text)
    text = convert_unicode(text) 
    text = standardize_sentence_typing(text)
    text = normalize_acronyms(text)
    # text = word_segmentation(text) # When use PhoBERT
    text = remove_unnecessary_characters(text)
    text = remove_redundant_newlines(text)
    # return text.lower()
    return text

In [28]:
df['processed_content'] = df['content'].apply(text_preprocess)

In [29]:
output_cols = ['id', 'title', 'content', 'processed_content', 'rating']
df = df[output_cols]
df

,id,title,content,processed_content,rating
0,9724102,Cực kì hài lòng,Gôd,gôd,5
1,19582868,Cực kì hài lòng,Hàng tốt,hàng tốt,5
2,17441069,Cực kì hài lòng,Bàn phím chất lượng như hình ảnh. Độ nãy tốt.\...,bàn phím chất lượng như hình ảnh độ nãy tốt xứ...,5
3,5373697,Rất không hài lòng,bàn phím loạn xị ngậu lên . chuột thì k sao cò...,bàn phím loạn xị ngậu lên chuột thì không sao ...,1
4,13821612,Cực kì hài lòng,Mình thấy ảnh sản phẩm màu đen nên tưởng sẽ nh...,mình thấy ảnh sản phẩm màu đen nên tưởng sẽ nh...,5
...,...,...,...,...,...
6510,19856146,Cực kì hài lòng,Âm thanh tốt,âm thanh tốt,5
6511,15249248,Cực kì hài lòng,"Đã nhận dc đồng hồ,Tiki giao hàng và tư vấn ch...",đã nhận được đồng hồ tiki giao hàng và tư vấn ...,5
6514,14210789,Cực kì hài lòng,"Chất lượng xem ra không được như mong đợi, tuy...",chất lượng xem ra không được như mong đợi tuy ...,3
6515,19600851,Cực kì hài lòng,Âm thanh tốt. Ko bị đau tai. Chống ổn tốt.,âm thanh tốt không bị đau tai chống ổn tốt,5


### Review label distribution

In [30]:
df['rating'].value_counts()

rating
5    3046
1     664
4     559
3     277
2     212
Name: count, dtype: int64

In [ ]:
df.loc[:, 'sentiment_3_cls'] = df['rating'].apply(lambda x: 'neutral' if x == 3 else 'negative' if x < 3 else 'positive')
df.loc[:, 'sentiment_2_cls'] = df['rating'].apply(lambda x: 'negative' if x < 3 else 'positive')

In [32]:
df

,id,title,content,processed_content,rating,sentiment_3_cls,sentiment_2_cls
0,9724102,Cực kì hài lòng,Gôd,gôd,5,positive,positive
1,19582868,Cực kì hài lòng,Hàng tốt,hàng tốt,5,positive,positive
2,17441069,Cực kì hài lòng,Bàn phím chất lượng như hình ảnh. Độ nãy tốt.\...,bàn phím chất lượng như hình ảnh độ nãy tốt xứ...,5,positive,positive
3,5373697,Rất không hài lòng,bàn phím loạn xị ngậu lên . chuột thì k sao cò...,bàn phím loạn xị ngậu lên chuột thì không sao ...,1,negative,negative
4,13821612,Cực kì hài lòng,Mình thấy ảnh sản phẩm màu đen nên tưởng sẽ nh...,mình thấy ảnh sản phẩm màu đen nên tưởng sẽ nh...,5,positive,positive
...,...,...,...,...,...,...,...
6510,19856146,Cực kì hài lòng,Âm thanh tốt,âm thanh tốt,5,positive,positive
6511,15249248,Cực kì hài lòng,"Đã nhận dc đồng hồ,Tiki giao hàng và tư vấn ch...",đã nhận được đồng hồ tiki giao hàng và tư vấn ...,5,positive,positive
6514,14210789,Cực kì hài lòng,"Chất lượng xem ra không được như mong đợi, tuy...",chất lượng xem ra không được như mong đợi tuy ...,3,neutral,positive
6515,19600851,Cực kì hài lòng,Âm thanh tốt. Ko bị đau tai. Chống ổn tốt.,âm thanh tốt không bị đau tai chống ổn tốt,5,positive,positive


In [33]:
df['sentiment_3_cls'].value_counts()

sentiment_3_cls
positive    3605
negative     876
neutral      277
Name: count, dtype: int64

In [34]:
df['sentiment_3_cls'].value_counts(normalize=True) * 100

sentiment_3_cls
positive    75.767129
negative    18.411097
neutral      5.821774
Name: proportion, dtype: float64

In [35]:
df.to_csv('processed_data/review_electronic_devices_20240511.csv', index=False)